In [2]:
!pip install usaddress
!pip install rapidfuzz

In [3]:
import pandas as pd
import usaddress
from rapidfuzz import process, fuzz

In [4]:
# Load the CSV files into Pandas dataframes
google_df = pd.read_csv('data/Google_Processed.csv')
tripadvisor_df = pd.read_csv('data/Trip_Advisor_Processed.csv')
yelp_df = pd.read_csv('data/Yelp_Processed.csv')

# Display the first few rows of each dataframe
google_df.head(), tripadvisor_df.head(), yelp_df.head()

(   Unnamed: 0                       Name             Main Tag  \
 0           0                 Mecca Cafe                Diner   
 1           1    Lost Lake Cafe & Lounge                Diner   
 2           2       Seattle Fish Company   Seafood restaurant   
 3           3  Locust Cider Market Place            Cider bar   
 4           5          Marination Ma Kai  Hawaiian restaurant   
 
                                                 Tags  Rating  Num Reviews  \
 0  Diner, Bar, Breakfast restaurant, Brunch resta...     4.2       1629.0   
 1  Diner, Bar, Breakfast restaurant, Brunch resta...     4.2       3837.0   
 2                                 Seafood restaurant     4.5        963.0   
 3                            Cider bar, Bar, Brewpub     4.7        108.0   
 4                           Hawaiian restaurant, Bar     4.5       3116.0   
 
                                              Address  Zip Code  \
 0  Mecca Cafe, 526 Queen Anne Ave N, Seattle, WA ...   98109.0  

In [5]:
# Check neighborhood columns
print(sorted(google_df['Neighborhood'].unique()))
print(sorted(tripadvisor_df['Neighborhood'].unique()))
print(sorted(yelp_df['Neighborhood'].unique()))

# Check dataset shape
print(google_df.shape)
print(tripadvisor_df.shape)
print(yelp_df.shape)
print(google_df.shape[0] + tripadvisor_df.shape[0] + yelp_df.shape[0])

['Ballard', 'Belltown', 'Bitter Lake', 'Capitol Hill', 'Chinatown', 'Downtown', 'Fremont/Wallingford', 'Northgate', 'Other', 'Queen Anne/South Lake Union', 'University District']
['Ballard', 'Belltown', 'Bitter Lake', 'Capitol Hill', 'Chinatown', 'Downtown', 'Fremont/Wallingford', 'Northgate', 'Other', 'Queen Anne/South Lake Union', 'University District']
['Ballard', 'Belltown', 'Bitter Lake', 'Capitol Hill', 'Chinatown', 'Downtown', 'Fremont/Wallingford', 'Northgate', 'Other', 'Queen Anne/South Lake Union', 'University District']
(2258, 9)
(2661, 12)
(2592, 10)
7511


In [6]:
# Function to extract street names
def extract_street_name(address):
    try:
        parsed_address = usaddress.parse(address)
        # Extract the street name from the parsed address components
        street_name = ' '.join([part[0] for part in parsed_address if 'StreetName' in part[1]])
        return street_name
    except usaddress.RepeatedLabelError as e:
        print(f"Error parsing address: {address}")
        return None

# Apply the function to the address columns
google_df['Street Name'] = google_df['Address'].apply(extract_street_name)
tripadvisor_df['Street Name'] = tripadvisor_df['Address'].apply(extract_street_name)
yelp_df['Street Name'] = yelp_df['Address'].apply(extract_street_name)

In [7]:
def get_best_match(row, target_df, column_name, scorer=fuzz.WRatio, score_cutoff=70):
    """
    Find the best match for a given name in a target DataFrame's column.
    
    Parameters:
    - row: The row of the source DataFrame.
    - target_df: DataFrame containing potential matches.
    - column_name: The name of the column in target_df to search for matches.
    - scorer: The RapidFuzz scoring function to use.
    - score_cutoff: The minimum score to consider a match (0-100).
    
    Returns:
    A tuple of the best match and its score, or None if no match above the cutoff.
    """
    best_match = process.extractOne(row[column_name], target_df[column_name], 
                                    scorer=scorer, score_cutoff=score_cutoff)
    return best_match[0] if best_match else row[column_name]

In [8]:
google_df['Name'] = google_df.apply(get_best_match, target_df=tripadvisor_df, column_name='Name', axis=1)


In [11]:
# Merge strategy based on the Street Name, Zip Code, and Name (fuzzy-match) as keys for merging

merged_df = pd.merge(google_df, tripadvisor_df, left_on=['Name', 'Zip Code', 'Street Name'], right_on=['Name', 'Zip Code', 'Street Name'], how='outer', suffixes=('_google', '_tripadvisor'))
merged_df['Name'] = merged_df.apply(get_best_match, target_df=yelp_df, column_name='Name', axis=1)
merged_df = pd.merge(merged_df, yelp_df, on=['Name', 'Zip Code', 'Street Name'], how='outer')
merged_df

,Unnamed: 0_google,Name,Main Tag,Tags_google,Rating_google,Num Reviews_google,Address_google,Zip Code,Neighborhood_google,Street Name,...,Tags_tripadvisor,Neighborhood_tripadvisor,Unnamed: 0,Address,Rating,Num Reviews,Cost_y,Tags,Distribution_y,Neighborhood
0,0.0,Mecca Cafe,Diner,"Diner, Bar, Breakfast restaurant, Brunch resta...",4.2,1629.0,"Mecca Cafe, 526 Queen Anne Ave N, Seattle, WA ...",98109.0,Queen Anne/South Lake Union,"Queen Anne Ave N,",...,AmericanBarDiner,Queen Anne/South Lake Union,129.0,"526 Queen Anne Ave N, Seattle, WA 98109",3.7,850.0,$$,"['American', 'Bars']","['118', '68', '105', '247', '312']",Queen Anne/South Lake Union
1,1.0,Lost Lake Cafe & Lounge,Diner,"Diner, Bar, Breakfast restaurant, Brunch resta...",4.2,3837.0,"Lost Lake Cafe & Lounge, 1505 10th Ave, Seattl...",98122.0,Capitol Hill,"10th Ave,",...,AmericanBarCafe,Capitol Hill,358.0,"1500 10th Ave, Seattle, WA 98122-3807",3.1,1136.0,$$,"['Breakfast & Brunch', 'New American', 'Beer B...","['252', '166', '175', '267', '276']",Capitol Hill
2,2.0,Seattle Fish Company,Seafood restaurant,Seafood restaurant,4.5,963.0,"Seattle Fish Company, 4435 California Ave SW, ...",98116.0,Other,"California Ave SW,",...,AmericanSeafood,Other,488.0,"4435 California Ave SW, Seattle, WA 98116",3.9,513.0,$$,"['Seafood Markets', 'Seafood', 'Fish & Chips']","['52', '42', '60', '110', '249']",Other
3,3.0,Locust Cider Market Place,Cider bar,"Cider bar, Bar, Brewpub",4.7,108.0,"Locust Cider Market Place, 2003 Western Ave Su...",98121.0,Belltown,Western Ave,...,AmericanPub,Belltown,2955.0,"2003 Western Ave Suite 110C, Seattle, WA 98121...",4.7,49.0,$$,"['Cideries', 'Brewpubs']","['0', '1', '2', '10', '36']",Belltown
4,5.0,Marination Ma Kai,Hawaiian restaurant,"Hawaiian restaurant, Bar",4.5,3116.0,"Marination Ma Kai, 1660 Harbor Ave SW, Seattle...",98126.0,Other,"Harbor Ave SW,",...,Quick BitesBarPolynesian,Other,91.0,"1660 Harbor Ave SW, Seattle, WA 98126-2028",4.1,1279.0,$$,"['Korean', 'Hawaiian', 'Asian Fusion']","['52', '69', '165', '408', '585']",Other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3528,NaN,The Pizza Source,NaN,NaN,NaN,NaN,NaN,98168.0,NaN,"Des Moines Memorial Dr S,",...,NaN,NaN,2931.0,"12008 Des Moines Memorial Dr S, Seattle, WA 98...",4.1,56.0,$,['Pizza'],"['4', '5', '4', '14', '29']",Other
3529,NaN,The Bubble Tea Shop,NaN,NaN,NaN,NaN,NaN,98109.0,NaN,Harrison St,...,NaN,NaN,3054.0,"305 Harrison St The Armory, Seattle, WA 98109-...",2.4,152.0,$,"['Bubble Tea', 'Juice Bars & Smoothies']","['73', '26', '12', '7', '34']",Queen Anne/South Lake Union
3530,NaN,Subway,NaN,NaN,NaN,NaN,NaN,98155.0,NaN,15th Avenue NE,...,NaN,NaN,3301.0,"18002 15th Avenue NE Suite A, Seattle, WA 98155",2.8,32.0,$,"['Sandwiches', 'Fast Food']","['12', '5', '2', '5', '8']",Other
3531,NaN,Taco Bell,NaN,NaN,NaN,NaN,NaN,98188.0,NaN,"Tukwila International Blvd.,",...,NaN,NaN,3324.0,"15036 Tukwila International Blvd., Seattle, WA...",1.7,24.0,$,"['Fast Food', 'Mexican']","['18', '0', '3', '1', '2']",Other


In [12]:
merged_df = merged_df.drop_duplicates(keep='last')
merged_df

,Unnamed: 0_google,Name,Main Tag,Tags_google,Rating_google,Num Reviews_google,Address_google,Zip Code,Neighborhood_google,Street Name,...,Tags_tripadvisor,Neighborhood_tripadvisor,Unnamed: 0,Address,Rating,Num Reviews,Cost_y,Tags,Distribution_y,Neighborhood
0,0.0,Mecca Cafe,Diner,"Diner, Bar, Breakfast restaurant, Brunch resta...",4.2,1629.0,"Mecca Cafe, 526 Queen Anne Ave N, Seattle, WA ...",98109.0,Queen Anne/South Lake Union,"Queen Anne Ave N,",...,AmericanBarDiner,Queen Anne/South Lake Union,129.0,"526 Queen Anne Ave N, Seattle, WA 98109",3.7,850.0,$$,"['American', 'Bars']","['118', '68', '105', '247', '312']",Queen Anne/South Lake Union
1,1.0,Lost Lake Cafe & Lounge,Diner,"Diner, Bar, Breakfast restaurant, Brunch resta...",4.2,3837.0,"Lost Lake Cafe & Lounge, 1505 10th Ave, Seattl...",98122.0,Capitol Hill,"10th Ave,",...,AmericanBarCafe,Capitol Hill,358.0,"1500 10th Ave, Seattle, WA 98122-3807",3.1,1136.0,$$,"['Breakfast & Brunch', 'New American', 'Beer B...","['252', '166', '175', '267', '276']",Capitol Hill
2,2.0,Seattle Fish Company,Seafood restaurant,Seafood restaurant,4.5,963.0,"Seattle Fish Company, 4435 California Ave SW, ...",98116.0,Other,"California Ave SW,",...,AmericanSeafood,Other,488.0,"4435 California Ave SW, Seattle, WA 98116",3.9,513.0,$$,"['Seafood Markets', 'Seafood', 'Fish & Chips']","['52', '42', '60', '110', '249']",Other
3,3.0,Locust Cider Market Place,Cider bar,"Cider bar, Bar, Brewpub",4.7,108.0,"Locust Cider Market Place, 2003 Western Ave Su...",98121.0,Belltown,Western Ave,...,AmericanPub,Belltown,2955.0,"2003 Western Ave Suite 110C, Seattle, WA 98121...",4.7,49.0,$$,"['Cideries', 'Brewpubs']","['0', '1', '2', '10', '36']",Belltown
4,5.0,Marination Ma Kai,Hawaiian restaurant,"Hawaiian restaurant, Bar",4.5,3116.0,"Marination Ma Kai, 1660 Harbor Ave SW, Seattle...",98126.0,Other,"Harbor Ave SW,",...,Quick BitesBarPolynesian,Other,91.0,"1660 Harbor Ave SW, Seattle, WA 98126-2028",4.1,1279.0,$$,"['Korean', 'Hawaiian', 'Asian Fusion']","['52', '69', '165', '408', '585']",Other
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3528,NaN,The Pizza Source,NaN,NaN,NaN,NaN,NaN,98168.0,NaN,"Des Moines Memorial Dr S,",...,NaN,NaN,2931.0,"12008 Des Moines Memorial Dr S, Seattle, WA 98...",4.1,56.0,$,['Pizza'],"['4', '5', '4', '14', '29']",Other
3529,NaN,The Bubble Tea Shop,NaN,NaN,NaN,NaN,NaN,98109.0,NaN,Harrison St,...,NaN,NaN,3054.0,"305 Harrison St The Armory, Seattle, WA 98109-...",2.4,152.0,$,"['Bubble Tea', 'Juice Bars & Smoothies']","['73', '26', '12', '7', '34']",Queen Anne/South Lake Union
3530,NaN,Subway,NaN,NaN,NaN,NaN,NaN,98155.0,NaN,15th Avenue NE,...,NaN,NaN,3301.0,"18002 15th Avenue NE Suite A, Seattle, WA 98155",2.8,32.0,$,"['Sandwiches', 'Fast Food']","['12', '5', '2', '5', '8']",Other
3531,NaN,Taco Bell,NaN,NaN,NaN,NaN,NaN,98188.0,NaN,"Tukwila International Blvd.,",...,NaN,NaN,3324.0,"15036 Tukwila International Blvd., Seattle, WA...",1.7,24.0,$,"['Fast Food', 'Mexican']","['18', '0', '3', '1', '2']",Other


In [13]:
merged_df.to_csv("CompleteRestaurantData.csv")